In [1]:
import os, pickle, sys
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import glob
from prettytable import PrettyTable
from tqdm import tqdm
from torch.utils.data import Dataset

import torch
from torch import nn
from torch.nn import Linear,Sequential
from torch.utils.data import DataLoader
import torch.nn.functional as func
import time
from math import isnan,isinf,log,exp
import random

In [18]:
fname=r'../myp/test.p'
rname='CIFAR10'

In [19]:
runs=[]
f = open(fname,'rb')
while(1):
    try:
        runs.append(pickle.load(f))
    except EOFError:
        break
f.close()
print(fname, len(runs))

../myp/test.p 10


In [21]:
runs[1]['logmeasures']['synflow']

[3.3873557890138598e+31,
 2.892258676489826e+31,
 0.0,
 4.2374668164538093e+30,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.48098050445203e+31,
 0.0,
 0.0,
 0.0,
 2.922203870601241e+31,
 0.0,
 4.081715009952707e+30,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.506482167061652e+31,
 0.0,
 0.0,
 0.0,
 2.897909889642198e+31,
 0.0,
 4.303844479119984e+30,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.479474742725912e+31,
 0.0,
 0.0,
 0.0,
 2.908833541844009e+31,
 0.0,
 4.2028317748093634e+30,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.488731887786098e+31,
 0.0,
 0.0,
 0.0,
 2.9187711429550664e+31,
 0.0,
 4.111950160943616e+30,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.5069812953811893e+31,
 0.0,
 0.0,
 0.0,
 3.3994575188969436e+31,
 1.910331463845202e+20,
 4.593125632969997e+20,
 3.035557915761409e+31,
 0.0,
 3.2497954779916575e+30,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.711824802020436e+31,
 0.0,
 0.0,
 0.0,
 3.0278789948402386e+31,
 0.0,
 3.316001123939683e+30,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.7056012726603134e+31,
 0.0,
 0.0,
 0.0,
 3.03137271567

In [6]:
import numpy as np

# 这个图是一个有向无环图(DAG)
# -1 代表什么也不连
# 图用二维列表，第一行代表节点编号，第二行为对应节点的指向节点
graph = [
    [0,0,1,2,3],
    [1,2,3,3,4]
]

# 定义5个节点的初始特征值
embeddings = [
    [1,2,3],
    [2,6,5],
    [2,3,7],
    [7,8,6],
    [1,0,0]
]

# 定义聚合的权重w全为1
w = [1,1,1,1,1]

# 下面开始图神经网络的聚合过程（训练过程）
# 在这里每个节点只按照方向聚合一层
for i in range(len(graph[0])):  # 每个节点
    # 先寻找指向节点i的节点们
    temp_roots = []
    for j, eve in enumerate(graph[1]):
        if eve == i:
            temp_roots.append(graph[0][j])
    temp_roots.append(i)
    # 此时temp_roots存储了节点i的根节点以及节点i自己的编号
    around = [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ]
    # 将temp_roots中的几点对应的around替换成当前的embedding
    for every_node_id in temp_roots:
        around[every_node_id] = embeddings[every_node_id]
    # 开始更新节点i的特征：自己之前的特征+周围节点特征的平均
    embeddings[i] = np.matmul(np.array(w),np.array(around))

# 输出更新一层后的embeddings
print(embeddings)

[array([1, 2, 3]), array([3, 8, 8]), array([ 3,  5, 10]), array([13, 21, 24]), array([14, 21, 24])]


In [7]:
torch.__version__

'1.12.1'

In [10]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
print(data)

Data(x=[3, 1], edge_index=[2, 4])
